# Psi4 Lab 02: Introduction


## Atomization Energy

## Part1

Give the number of basis functions and primitive Gaussians for Hydrogen, Nitrogen, Oxygen and Carbon atoms using STO-3G, 6-31G(d), and 6-311+G(2d,p) basis sets.  Briefly comment on your finding. (i.e. how many basis functions, types of basis functions, how does the calculated energy varied).

In [8]:
import psi4

psi4.set_memory('1 GB')

1000000000

In [9]:
import psi4

hatom = psi4.geometry("""
0 2
H
""")

psi4.set_options({'reference': 'uhf'})
psi4.core.set_output_file('h-sto3g.log', False)
eng = psi4.energy('hf/sto-3g', molecule=hatom)
print(eng)      

-0.4665818495572755


In [3]:
!cat h-sto3g.log


Scratch directory: /tmp/

Scratch directory: /tmp/

*** tstart() called on nohostname
*** at Tue Mar 30 04:21:58 2021

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry H          line    19 file /srv/conda/envs/notebook/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UHF Reference
                        1 Threads,    953 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: d2h
    Geometry (in Angstrom), charge = 0, multiplicity = 2:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         H          

In [10]:
psi4.set_options({'reference': 'uhf'})
psi4.core.set_output_file('h-631gs.log', False)
eng = psi4.energy('hf/6-31G*', molecule=hatom)
print(eng)

-0.4982329107290702


In [5]:
!cat h-631gs.log


Scratch directory: /tmp/

Scratch directory: /tmp/

*** tstart() called on nohostname
*** at Tue Mar 30 04:22:07 2021

   => Loading Basis Set <=

    Name: 6-31G*
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry H          line    44 file /srv/conda/envs/notebook/share/psi4/basis/6-31gs.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UHF Reference
                        1 Threads,    953 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: d2h
    Geometry (in Angstrom), charge = 0, multiplicity = 2:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         H          

In [11]:
psi4.set_options({'reference': 'uhf'})
psi4.core.set_output_file('h-6311g2dp.log', False)
eng = psi4.energy('hf/6-311G(2d,p)', molecule=hatom)
print(eng)

-0.49980981527328194


In [7]:
!cat h-6311g2dp.log


Scratch directory: /tmp/

Scratch directory: /tmp/

*** tstart() called on nohostname
*** at Tue Mar 30 04:22:15 2021

   => Loading Basis Set <=

    Name: 6-311G(2D,P)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1 entry H          line    40 file /srv/conda/envs/notebook/share/psi4/basis/6-311g_2d_p_.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UHF Reference
                        1 Threads,    953 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: d2h
    Geometry (in Angstrom), charge = 0, multiplicity = 2:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
        

Now try to calculate Nitrogen (ground state "quartet S"), Oxygen (ground state "triplet P") and Carbon (ground state "triplet P")

https://quantummechanics.ucsd.edu/ph130a/130_notes/node393.html
https://quantummechanics.ucsd.edu/ph130a/130_notes/node394.html
https://quantummechanics.ucsd.edu/ph130a/130_notes/node392.html

In [ ]:
# carbon in STO-3G

In [ ]:
# carbon in 6-31G*

In [ ]:
# carbon in 6-311G(2d,p)

In [ ]:
# Oxygen in STO-3G

In [ ]:
# Oxygen in 6-31G*

In [ ]:
# Oxygen in 6-311G(2d,p)

In [ ]:
# Nitrogen in STO-3G

In [ ]:
# Nitrogen in 6-31G*

In [ ]:
# Nitrogen in 6-311G(2d,p)

## Or you can construct the geometry as the following to specify the geometries

In [12]:
charge = 0
multp = 1
rHH = 1.0 # Ångstrom
geom = f'{charge} {multp}\nH\nH 1 {rHH:.6f}' # we use formatted strings
print(geom)

0 1
H
H 1 1.000000


In [13]:
mol = psi4.geometry(geom)

In [14]:
print(f"Number of atoms = {mol.natom()}")
print(f"Nuclear repulsion energy = {mol.nuclear_repulsion_energy()}")

Number of atoms = 2
Nuclear repulsion energy = 0.5291772106699999


In [15]:
basis = 'cc-pVDZ'
reference = 'rhf'

psi4.core.clean()

psi4.set_options({'basis': basis,'scf_type': 'pk', 'reference' : reference})

# pipe output to the file output.dat
psi4.core.set_output_file('output.dat', False)
# run scf and return the energy and a wavefunction object (will work only if pass return_wfn=True)
E_scf, wfn = psi4.energy('scf', return_wfn=True)

In [14]:
print(f"Number of orbitals = {wfn.nmo()}")
print(f"Number of alpha electrons = {wfn.nalpha()}")
print(f"Number of irreducible representations (irreps) = {wfn.nirrep()}")
nirrep = wfn.nirrep()

Number of orbitals = 10
Number of alpha electrons = 1
Number of irreducible representations (irreps) = 8


## Part2

Give the number of basis functions, number of primitive Gaussians, number of alpha electrons, and beta electrons of H2O, NH3, CH4, and C2H6 using STO-3G, 6-31G(d), and 6-311+G(2d,p) basis sets.  Briefly comment on the geometry you used (i.e. RCH and AHCH …)

In [16]:
h2o_zmat = psi4.geometry("""
0 1
O
H 1 1.0
H 1 1.0 2 104.5
""")

print(h2o_zmat.to_string(dtype='xyz'))

3
0 1 H2O
O                     0.000000000000     0.000000000000    -0.068516219320
H                     0.000000000000    -0.790689573744     0.543701060715
H                     0.000000000000     0.790689573744     0.543701060715



In [17]:
# pipe output to the file output.dat
psi4.core.set_output_file('h2o-sto3g.log', False)
psi4.optimize('scf/sto-3g', molecule=h2o_zmat)

Optimizer: Optimization complete!


-74.96590116867525

In [18]:
#print optimized geometry
print(h2o_zmat.to_string(dtype='xyz'))

3
0 1 H2O
O                     0.000000000000     0.000000000000    -0.071174078038
H                     0.000000000000    -0.758045150859     0.564792134020
H                     0.000000000000     0.758045150859     0.564792134020



In [19]:
!cat h2o-sto3g.log


Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on nohostname
*** at Tue Mar 30 04:25:49 2021

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /srv/conda/envs/notebook/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /srv/conda/envs/notebook/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    953 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center          

In [20]:
# pipe output to the file output.dat
psi4.core.set_output_file('h2o-631Gs.log', False)
psi4.optimize('scf/6-31G*', molecule=h2o_zmat)

Optimizer: Optimization complete!


-76.01074650534503

In [21]:
#print optimized geometry
print(h2o_zmat.to_string(dtype='xyz'))

3
0 1 H2O
O                     0.000000000000     0.000000000000    -0.064176373753
H                     0.000000000000    -0.754011416453     0.509262811475
H                     0.000000000000     0.754011416453     0.509262811475



In [38]:
!cat h2o-631Gs.log


Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on nohostname
*** at Tue Mar 30 04:02:57 2021

   => Loading Basis Set <=

    Name: 6-31G*
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   145 file /srv/conda/envs/notebook/share/psi4/basis/6-31gs.gbs 
    atoms 2-3 entry H          line    44 file /srv/conda/envs/notebook/share/psi4/basis/6-31gs.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UHF Reference
                        1 Threads,    953 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center          

In [40]:
# pipe output to the file output.dat
psi4.core.set_output_file('h2o-6311G2dp.log', False)
psi4.optimize('scf/6-311G(2d,p)', molecule=h2o_zmat)

Optimizer: Optimization complete!


-76.0481910315822

In [41]:
#print optimized geometry
print(h2o_zmat.to_string(dtype='xyz'))

3
0 1 H2O
O                     0.000000000000     0.000000000000    -0.063764666779
H                     0.000000000000    -0.749974548368     0.505995767249
H                     0.000000000000     0.749974548368     0.505995767249



In [42]:
!cat h2o-6311G2dp.log


Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on nohostname
*** at Tue Mar 30 04:03:10 2021

   => Loading Basis Set <=

    Name: 6-311G(2D,P)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   169 file /srv/conda/envs/notebook/share/psi4/basis/6-311g_2d_p_.gbs 
    atoms 2-3 entry H          line    40 file /srv/conda/envs/notebook/share/psi4/basis/6-311g_2d_p_.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              UHF Reference
                        1 Threads,    953 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c2v
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

     

Continue for  NH3, CH4, and C2H6

In [ ]:
# NH3 in STO-3G

In [ ]:
# NH3 in 6-31G*

In [ ]:
# NH3 in 6-311G(2d,p)

In [ ]:
# CH4 in STO-3G

In [ ]:
# CH4 in 6-31G*

In [ ]:
# CH4 in 6-311G(2d,p)

In [ ]:
#C2H6 in STO-3G

In [ ]:
#C2H6 in 6-31G*

In [ ]:
#C2H6 in 6-311G(2d,p)

## Part3	

Tabulate the electronic energies using Hartree Fock method and calculate the atomization energy. 

(i.e.  AE(CH4) = E(CH4) – E(C) – 4* E(H))

Method |	E(H) |	E(C)   |	E(O)  |	E(CH4)  | AE(CH4)
-------|---------|---------|----------|---------|---------
HF/STO-3G  |     |      |      |      |								
HF/6-31G(d)   |    |     |     |      |								
HF/6-311+G(2d,p)    |    |     |      | 


   
   1 Eh (Hartree) = 627.5096 kcal/mol
   Literature value of Atomization energies taken from CCCBDB (http://cccbdb.nist.gov/expdata.asp)
   
	 H2O: 219.4 kcal/mol (0K)    221.6 kcal/mol (298 K)
	 NH3: 276.7 kcal/mol (0K)    280.3 kcal/mol (298 K)
	 CH4 : 392.4 kcal/mol (0K)    397.5 kcal/mol (298K)
	 C2H6: 666.1 kcal/mol (0K)    675.3 kcal/mol (298 K)


In [ ]:
|